# Preparing the data for inference
This notebook contains the pipeline for downloading and preparing the data for inference.
The data for training can also be downloaded and prepared using this pipeline.

In [ ]:
import satsearch
import os
import rasterio
import pyproj
from osgeo import gdal
import requests
import tqdm
import geopandas as gpd
import utils
import numpy as np
from rasterio.merge import merge

### Defining the area of interest

In [ ]:
# select the city name 
city_name = "Hannover"

# getting bbox coordinates from multipolygon in administrative boundaries geojson file
filepath_json = f"path/{city_name}/{city_name}_ab.geojson"
coords = utils.get_bbox_from_geojson(filepath_json)
print(coords) # coordinates in WGS84 used for querying the STAC

# getting coordinates from geojson
polygon_gdf = gpd.read_file(filepath_json)

(9.604728, 52.305911, 9.91856, 52.453728)


In [ ]:
# dict with city names and their respective bboxes, Sentinel-2 tile ids and EPSG coordinate codes
# follow the dict structure to add/change cities
city_dicts = {
            "Bremen": {
                "bbox": [8.481735, 53.011021, 8.990913, 53.597327],
                "tile_id": "32UMD",
                "epsg": 32632},
            "Bielefeld": {
                "bbox": [8.377677, 51.91497, 8.663542, 52.114833],
                "tile_id": "32UMC",
                "epsg": 32632},
            "Essen": {
                "bbox": [6.894508, 51.347413, 7.136842, 51.534127],
                "tile_id": "32ULC",
                "epsg": 32632},
            "Duesseldorf": {
                "bbox": [6.689823, 51.124394, 6.939965, 51.351446],
                "tile_id": "32ULB",
                "epsg": 32632},
            "Cologne": {
                "bbox": [6.77256, 50.830434, 7.162043, 51.084961], 
                "tile_id": "31UGS",
                "epsg": 32631},
            "Frankfurt": {
                "bbox": [8.472633, 50.015574, 8.800535, 50.226257],
                "tile_id": "32UMA",
                "epsg": 32632},
            "Nuremberg": {
                "bbox": [10.987928, 49.331937, 11.21357, 49.540333],
                "tile_id": "32UPV",
                "epsg": 32632},
            "Dresden": {
                "bbox": [13.579721, 50.975184, 13.965849, 51.177704],
                "tile_id": "33UVS",
                "epsg": 32633},
            "Leipzig": {
                "bbox": [12.236779, 51.238154, 12.542607, 51.448067],
                "tile_id": ["32UQB", "32UQC"],
                "epsg": 32632},
            "Hannover": {
                "bbox": [9.604728, 52.305911, 9.91856, 52.453728],
                "tile_id": "32UND",
                "epsg": 32632},
            "Munich": {
                "bbox": [11.360877, 48.061554, 11.723083, 48.248146],
                "tile_id": "32UPU",
                "epsg": 32632},
            "Berlin": {
                "bbox": [13.088333, 52.338242, 13.760469, 52.675379],
                "tile_id": ["33UVU", "33UUU"],
                "epsg": 32633},
            "Hamburg": {
                "bbox": [9.734365, 53.39507, 10.325959, 53.738472],
                "tile_id": ["32UNE"],
                "epsg": 32632},
            "Dortmund": {
                "bbox": [7.302311, 51.41558, 7.637933, 51.59952],
                "tile_id": ["32ULC"],
                "epsg": 32632},
            "Stuttgart": {
                "bbox": [9.038994, 48.689956, 9.315387, 48.867249],
                "tile_id": ["32UNV"],
                "epsg": 32632},
            }


In [ ]:
# rasterizing the administrative boundaries geojson
rasterized_json_filepath, _ = os.path.splitext(filepath_json)
rasterized_json_filepath = f"{rasterized_json_filepath}.{".tif".lstrip('.')}"
utils.reproject_and_rasterize(filepath_json, f"EPSG:{city_dicts[city_name]["epsg"]}", rasterized_json_filepath)

In [ ]:
# creating directories for the selected city
dir_list = ["cogs", "composites", "tiles"]
path = "root/path"  

for dir in dir_list:
    if not os.path.exists(os.path.join(path, city_name, dir)):
        os.makedirs(os.path.join(path, city_name, dir))
        print("Directory created!")
    else:
        print("Directory already exists! Check city name.")

### Searching the STAC catalogue for the selected AOI

In [ ]:
bounding_box = city_dicts[city_name]["bbox"]

search = satsearch.Search(
    bbox = [i for i in bounding_box],
    datetime='2018-05-01/2018-09-30',
    collections=['sentinel-s2-l2a-cogs'],
    url='https://earth-search.aws.element84.com/v0' # Element84 public STAC catalogue
    )

print('bbox search: %s items' % search.found())

items = search.items()
#print(items.summary(['date', 'id', 'eo:cloud_cover']))

bbox search: 120 items


*During this step it is recommended to manually download each product to check for artifacts and clouds above AOI*

In [ ]:
bands = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B11', 'B12']
links_dict = {}
for i in items:
  if i['eo:cloud_cover'] <= 5.0 and i['eo:cloud_cover'] != 0: 
    for tile_id in city_dicts[city_name]["tile_id"]:
      if tile_id in i.id:
        links_dict[i.id] = []
        for band in bands:
          links_dict[i.id].append(i.asset(band)["href"])
      else: 
        continue
  else:
    continue
print("Number of products:", len(links_dict.keys()))
links_dict # returning the dict of links

In [ ]:
# filtering the links dict for chosen products
# products are chosen after manually downloading and inspecting them for cloud cover and artifacts
chosen_products_dict = {"Bremen": "S2B_32UMD_20180629_0_L2A",
                        "Bielefeld": "S2A_32UMC_20180806_0_L2A",
                        "Essen": "S2A_32ULC_20180707_0_L2A",
                        "Duesseldorf": "S2A_32ULB_20180707_0_L2A",
                        "Cologne": "S2B_31UGS_20180722_0_L2A",
                        "Frankfurt": "S2B_32UMA_20180818_0_L2A",
                        "Nuremberg": "S2A_32UPV_20180820_0_L2A",
                        "Dresden": "S2B_33UVS_20180514_0_L2A",
                        "Leipzig": ["S2B_32UQB_20180703_0_L2A", "S2B_32UQC_20180703_0_L2A"], # 2 products because AOI is covered by 2 products
                        "Hannover": "S2A_32UND_20180724_0_L2A",
                        "Munich": "S2A_32UPU_20180731_0_L2A",
                        "Berlin": ["S2A_33UVU_20180909_0_L2A", "S2A_33UUU_20180909_0_L2A"], # 2 products because AOI is covered by 2 products
                        "Hamburg": "S2A_32UNE_20180505_0_L2A",
                        "Dortmund": "S2A_32ULC_20180607_0_L2A",
                        "Stuttgart": "S2A_32UNV_20180505_0_L2A",
                        }

# creating the dict of links for each product band
links_dict = {key:value for (key, value) in links_dict.items() if key in chosen_products_dict[city_name]}
links_dict

### Downloading bands rasters and cropping them to the AOI (bbox)

In [ ]:
coords = utils.get_bbox_geotiff(rasterized_json_filepath)
bbox_epsg = [coords[0], coords[3], coords[2], coords[1]]
print(bbox_epsg)

In [ ]:
# function to crop one raster
def crop_one_raster (filepath, bounding_box, delete=False):
  window = (bounding_box[0], bounding_box[1], bounding_box[2], bounding_box[3])
  new_name = filepath[:-4] + '_cropped.tif'
  print(new_name)
  ds = gdal.Open(filepath)
  gdal.Translate(new_name, filepath, projWin = window)
  # deletion may not work in some systems, use with caution
  if delete:
    # rewriting file to 0 bytes
    open(filepath, 'w').close()
    # moving the 0-bytes file to bin
    os.remove(filepath)

In [ ]:
# downloading and cropping rasters
download_path = f'path/{city_name}/cogs/'

# creating directories if they don't exist
if not os.path.exists(download_path):
    os.makedirs(download_path)

for item in links_dict:
  filename = item
  for link, band in zip(links_dict[item], bands):
    filepath = os.path.join(download_path, f"{filename}_{band}.tif")
    r = requests.get(link, stream = True)
    print(filepath)
    with open(filepath, "wb") as file:
      for block in tqdm.tqdm(r.iter_content(chunk_size = 1024)):
        if block:
          file.write(block)
    input_path = filepath
    crop_one_raster(input_path, bbox_epsg, delete=False) # if delete=True original files are deleted

# deleting uncropped rasters
list_files = os.listdir(download_path)
list_files = [i for i in os.listdir(download_path) if "cropped" not in i]
for i in list_files:
    os.remove(os.path.join(download_path, i))

print("Download finished!")

### Creating all-bands composite

If not using automatic deletion for crop_one_raster function, make sure the downloaded original files are deleted, otherwise the following algorithm may results in errors.

In [ ]:
# creating dict of filenames in each band folder
filepaths_all = download_path
filepath_composites = f"path/{city_name}/composites/"

if not os.path.exists(filepath_composites):
    os.makedirs(filepath_composites)

# getting the list of files and sorting it
files_list = os.listdir(download_path)
if len(files_list) == 12:
  # sorting list (so the band 8A is after band 8)
  last_element = files_list.pop()
  files_list.insert(8, last_element)

  counter = 0
  opened_bands = []
  # making composites
  for i in range(0, len(files_list)):
    with rasterio.open(os.path.join(filepaths_all, files_list[i]), 'r') as src:
        if i == 1:
          band2_profile = src.profile
          band2_profile.update({"count": 12}) # increasing the band count to 12
        opened_bands.append(src.read(1))

  img_path = os.path.join(filepath_composites, f"{files_list[0][:24]}_composite.tif")

  with rasterio.open(img_path, 'w', **band2_profile) as img:
    counter = 1
    for band in opened_bands:
      img.write(band, counter)
      counter += 1

  # rewriting the geotiff to match the administrative boundaries
  utils.modify_geotiff_with_mask(img_path, rasterized_json_filepath, img_path)


# if folder contains more than 2 products
if len(files_list) > 12:
  # sorting lists and splitting them to list of lists
  list_of_files_list = []
  counter_ = 0
  for idx in range(1, int(len(files_list) / 12) + 1):
    modifier_1 = 11 + (counter_ * 12)
    modifier_2 = 8 + (counter_ * 12)
    last_element = files_list.pop(modifier_1)
    files_list.insert(modifier_2, last_element)
    counter_ += 1
    list_of_files_list.append(files_list[(modifier_1 - 11):(modifier_1 + 1)])
  
  # looping over products
  for files_list in list_of_files_list:
      counter = 0
      opened_bands = []
      for i in range(0, len(files_list)):
        with rasterio.open(os.path.join(filepaths_all, files_list[i]), 'r') as src:
            if i == 1:
              band2_profile = src.profile
              band2_profile.update({"count": 12}) # increasing the layer count to 12
            opened_bands.append(src.read(1))

      img_path = os.path.join(filepath_composites, f"{files_list[0][:24]}_composite.tif")

      with rasterio.open(img_path, 'w', **band2_profile) as img:
        counter = 1
        for band in opened_bands:
          img.write(band, counter)
          counter += 1

print("Composites created!")

In [ ]:
# merging composites together if there's more than one file in the composites folder 
list_comps = [i for i in os.listdir(filepath_composites) if "merged" not in i]
if len(list_comps) > 1:
    composites_list = []
    for comp in list_comps:
        src = rasterio.open(os.path.join(filepath_composites, comp))
        composites_list.append(src)
        out_meta = src.meta.copy()
    
    merged_comp, out_transform = merge(composites_list)

    for src in composites_list:
        src.close()

    out_meta.update({"driver": "GTiff",
                "height": merged_comp.shape[1],
                "width": merged_comp.shape[2],
                "transform": out_transform,
                #"crs": crs
                })

    path_merged_comp = os.path.join(filepath_composites, list_comps[0])
    path_merged_comp, _ = os.path.splitext(path_merged_comp)
    path_merged_comp = f"{path_merged_comp}_merged.{".tif".lstrip('.')}"

    with rasterio.open(path_merged_comp, "w", **out_meta) as dst:
        dst.write(merged_comp)

    # deleting original composites
    list_files = os.listdir(filepath_composites)
    list_files = [i for i in os.listdir(filepath_composites) if "merged" not in i]
    for i in list_files:
        os.remove(os.path.join(filepath_composites, i))
    
    list_files = os.listdir(filepath_composites)
    img_path = os.path.join(filepath_composites, list_files[0])
    utils.modify_geotiff_with_mask(img_path, rasterized_json_filepath, img_path)
    print("Processing finished!")

else:
    print("Only 1 composite is present in the folder! Nothing was done.")

## Tiling the composites
Splitting composites to tiles is necessary because the model only takes images of size (128, 128, 12). If the model input size is changed, the size should also be changed in the code below.

In [ ]:
# Defining the cropping function
def get_tiles(ds, tile_width, tile_height, overlap=0, check_zeroes=True):
    """
    Yields rasterio.window and the respective transform object.
    Args:
        ds: rasterio-opened datasource
        tile_width: width of the tile to yield
        tile_height: height of the tile to yield
        overlap: tile overlap (use 0 for inference data preparation)
        check_zeroes (bool): if True, omits every tile that consists only of zeroes. useful when AOI is not of rectangular shape
    """
    ncols, nrows = ds.meta['width'], ds.meta['height']
    xstep = tile_width - overlap
    ystep = tile_height - overlap
    for x in range(0, ncols, xstep):
        for y in range(0, nrows, ystep):
            window = rasterio.windows.Window(x, y, tile_width, tile_height)
            if check_zeroes:
                all_zero = np.all(src.read(1, window=window) == 0)
                if all_zero:
                    continue
                else:
                    transform = rasterio.windows.transform(window, ds.transform)
                    yield window, transform
            else:
                transform = rasterio.windows.transform(window, ds.transform)
                yield window, transform
                

In [ ]:
# batch tiling the images

# set tiling options
tile_width = 128 
tile_height = 128 
overlap = 0 

in_path = f"path/{city_name}/composites" # filepath to composites
out_path = f"path/{city_name}/tiles/" # filepath to output tiles

if not os.path.exists(in_path):
    os.makedirs(in_path)

output_filename = '{}_{}_{}.tif'
input_filenames = os.listdir(in_path)

for input_filename in input_filenames:
  with rasterio.open(os.path.join(in_path, input_filename)) as src:
    metadata = src.meta.copy()

    for window, transform in get_tiles(src, tile_width, tile_height, overlap):
        metadata['transform'] = transform
        metadata['width'], metadata['height'] = window.width, window.height
        out_filepath = os.path.join(out_path, output_filename.format(input_filename[:-17], window.col_off, window.row_off))

        with rasterio.open(out_filepath, 'w', **metadata) as dst:
            dst.write(src.read(window=window))
    print(out_filepath)
print("Tiling finished!")
print(len(os.listdir(out_path)))